# Final LDA Model

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

from datetime import datetime

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
# import pyLDAvis.gensim  # Author: don't skip this
# pyLDAvis.gensim.prepare

# I think i need another one:
import pyLDAvis.gensim_models
import pyLDAvis.gensim_models as gensimvis

# Plots
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('dutch')
stop_words.extend(['tenlastelegging\\n\\n\\naan', '\\n\\n\\n\\n\\n\\n', '\\n\\n\\n\\n\\n', '\\n\\n', 'een', 'naan', 'nde', 'nhij', '\\n'])

# Data Final

In [109]:
df = pd.read_csv('/Users/dj/Python - UvA/DSP/data/dataset_10012022_cleaned.csv', index_col=6, parse_dates=['date']) 
df.head()

df_be = df[df['bewijs'] != '[]'] #bewijs
df_te = df[df['tll'] != '[]'] #tenlastelegging

df['bete'] = df['bewijs'] + df['tll'] 
df = df[df['bete'] != '[][]']
df = df.drop(labels=['bewijs', 'tll'], axis=1)


In [110]:
df18 = df.loc["2018-01-01 00:00:00":"2018-12-31 00:00:00"]
df19 = df.loc["2019-01-01 00:00:00":"2019-12-31 00:00:00"]
df20 = df.loc["2020-01-01 00:00:00":"2020-12-31 00:00:00"]
df21 = df.loc["2021-01-01 00:00:00":"2021-12-31 00:00:00"]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated 

In [124]:
print(df18.shape)
print(df19.shape)
print(df20.shape)
print(df21.shape)

(300, 7)
(306, 7)
(315, 7)
(527, 7)


# Build Giant Function

In [157]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out
        

def get_lda_model(df):
    data_words = list(sent_to_words(df['bete']))

    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    # Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    # nlp = spacy.load('nl', disable=['parser', 'ner'])

    nlp = spacy.load("nl_core_news_sm")

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]
    # corpus1 = [str(item) for item in corpus]

    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=5, 
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

    return(lda_model, data_lemmatized, corpus, id2word)

In [158]:
d = [df18, df19, df20, df21]
q = 1
lda_models = []
dat_lems = []
corps = []
ids = []

for i in d:
    print(q)
    q += 1
    l, dl, c, i = get_lda_model(i)
    lda_models.append(l)
    dat_lems.append(dl)
    corps.append(c)
    ids.append(i)



1
2
3
4


In [156]:
d[3]

,Unnamed: 0,filename,ecli,subject,spatial,case_nr,bete
date,,,,,,,
2021-02-10,7,ECLI_NL_RBOVE_2021_606.xml,ECLI:NL:RBOVE:2021:606,Strafrecht,Almelo,08/760218-17 (P),"[][' tenlastelegging de verdenking komt er, ..."
2021-02-10,8,ECLI_NL_GHSHE_2021_1801.xml,ECLI:NL:GHSHE:2021:1801,Strafrecht,'s-Hertogenbosch,20-003457-18,[' bewijsmiddelen indien tegen dit ve...
2021-02-09,16,ECLI_NL_RBROT_2021_2966.xml,ECLI:NL:RBROT:2021:2966,Strafrecht,Rotterdam,10.295806.20,[' bewijswaardering 4.1.1. standpunt verd...
2021-02-02,28,ECLI_NL_RBAMS_2021_378.xml,ECLI:NL:RBAMS:2021:378,Strafrecht,Amsterdam,13/701115-19,[][' 2tenlastelegging aan verdachte is – ko...
2021-02-23,43,ECLI_NL_GHDHA_2021_497.xml,ECLI:NL:GHDHA:2021:497,Strafrecht,Den Haag,2200469519,[' bewijsvoering het hof grondt zijn ...
...,...,...,...,...,...,...,...
2021-02-03,2882,ECLI_NL_RBROT_2021_1286.xml,ECLI:NL:RBROT:2021:1286,Strafrecht,Rotterdam,10/750172-20,[][' tenlastelegging aan de verdachte word...
2021-02-25,2898,ECLI_NL_RBZWB_2021_786.xml,ECLI:NL:RBZWB:2021:786,Strafrecht,Middelburg,02-220946-20,[][' tenlastelegging de tenlastelegging is ge...
2021-02-25,2902,ECLI_NL_RBNHO_2021_1491.xml,ECLI:NL:RBNHO:2021:1491,Strafrecht,Haarlem,15/246961-19,[][' 1tenlastelegging aan verdachte is ten l...


# Compute Model Perplexity and Coherence Score

In [166]:
# 18 = 0 / 19 = 1 / 20 = 2 / 21 = 3

a = 3

# Compute Perplexity
print('\nPerplexity: ', lda_models[a].log_perplexity(corps[a]))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_models[a], texts=dat_lems[a], dictionary=ids[a], coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.126905997290772

Coherence Score:  0.4254388885067482


# Visualize the topics-keywords

In [162]:
# 18 = 0 / 19 = 1 / 20 = 2 / 21 = 3

a = 3

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_models[a], corps[a], ids[a], R = 10)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.043129 -0.060756       1        1  45.349316
3     -0.197275 -0.009482       2        1  19.700065
0      0.099694 -0.144908       3        1  14.537534
1      0.107685  0.038179       4        1  10.576052
2      0.033024  0.176967       5        1   9.837033, topic_info=             Term         Freq        Total Category  logprob  loglift
581   slachtoffer  6909.000000  6909.000000  Default  10.0000  10.0000
28           naam  5042.000000  5042.000000  Default   9.0000   9.0000
4         althans  2442.000000  2442.000000  Default   8.0000   8.0000
1779     kenteken  1171.000000  1171.000000  Default   7.0000   7.0000
691       bedrijf  1180.000000  1180.000000  Default   6.0000   6.0000
...           ...          ...          ...      ...      ...      ...
32        periode   272.750680   997.510318   Topic5  -4.8118   1.0223
581   slachtoffer   435.866482  6909.459193   Topic5  -4.3431  -0.4443
203         weten   280.231505  1933.350502   Topic5  -4.7848   0.3876
4         althans   269.737299  2442.685664   Topic5  -4.8229   0.1156
6           ander   246.469160  2617.053963   Topic5  -4.9132  -0.0436

[140 rows x 6 columns], token_table=      Topic      Freq              Term
term                                   
456       1  0.911637       aangeefster
456       2  0.087399       aangeefster
211       1  0.167600        aantreffen
211       2  0.042922        aantreffen
211       3  0.622367        aantreffen
...     ...       ...               ...
443       2  0.026017              zien
443       3  0.071954              zien
443       4  0.002846              zien
443       5  0.004472              zien
5860      1  0.991508  zover_inhoudenen

[296 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 2, 3])